In [1]:
from IPython.display import display, HTML

# Interfaz unificada: Cámara + OpenCV.js + Método de Sobel (Sin Canny)
opencv_realtime_sobel = """
<div style="display: flex; flex-direction: column; align-items: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif; width: 450px; margin: auto;">
    <h3 style="margin-bottom: 10px;">OpenCV Live: Método Sobel</h3>
    <div id="status" style="color: #ffcc00; margin-bottom: 10px;">Cargando OpenCV.js...</div>
    
    <video id="videoInput" width="400" height="300" autoplay playsinline style="display:none;"></video>
    
    <canvas id="canvasOutput" width="400" height="300" style="border: 2px solid #007bff; border-radius: 10px; background: #000;"></canvas>
    
    <div style="margin-top: 15px; font-size: 0.8em; color: #888; text-align: center;">
        <p>Calculando Magnitud del Gradiente en tiempo real:</p>
        <code>G = sqrt(Gx² + Gy²)</code>
    </div>
</div>

<script async src="https://docs.opencv.org/4.5.4/opencv.js" type="text/javascript"></script>

<script>
    const status = document.getElementById('status');
    const video = document.getElementById('videoInput');

    // Función que inicia cuando OpenCV.js está cargado en el navegador
    function onOpenCvReady() {
        status.innerHTML = "OpenCV Listo. Iniciando Cámara...";
        status.style.color = "#28a745";
        startVideo();
    }

    // Verificar si OpenCV ya cargó (por si el evento 'async' falla)
    let checkCv = setInterval(() => {
        if (typeof cv !== 'undefined' && cv.Mat) {
            onOpenCvReady();
            clearInterval(checkCv);
        }
    }, 1000);

    async function startVideo() {
        try {
            const stream = await navigator.mediaDevices.getUserMedia({ video: true, audio: false });
            video.srcObject = stream;
            video.play();
            processVideo();
        } catch (err) {
            status.innerHTML = " Error: Permiso de cámara denegado";
            status.style.color = "#ff4444";
        }
    }

    function processVideo() {
        // Inicializar matrices de OpenCV
        let src = new cv.Mat(video.height, video.width, cv.CV_8UC4);
        let dst = new cv.Mat(video.height, video.width, cv.CV_8UC1);
        let cap = new cv.VideoCapture(video);

        let grad_x = new cv.Mat();
        let grad_y = new cv.Mat();
        let abs_grad_x = new cv.Mat();
        let abs_grad_y = new cv.Mat();

        function process() {
            try {
                // Leer el cuadro actual de la cámara
                cap.read(src);
                
                // 1. Convertir a Gris (Requisito para derivadas)
                cv.cvtColor(src, dst, cv.COLOR_RGBA2GRAY);
                
                // 2. MÉTODOS NUMÉRICOS: Sobel (Derivadas parciales)
                // Calculamos Gx (Derivada horizontal)
                cv.Sobel(dst, grad_x, cv.CV_16S, 1, 0, 3);
                cv.convertScaleAbs(grad_x, abs_grad_x);

                // Calculamos Gy (Derivada vertical)
                cv.Sobel(dst, grad_y, cv.CV_16S, 0, 1, 3);
                cv.convertScaleAbs(grad_y, abs_grad_y);

                // 3. Combinar gradientes (Aproximación de la magnitud total)
                // Aumentamos el peso a 1.0 para que la silueta sea brillante
                cv.addWeighted(abs_grad_x, 1.0, abs_grad_y, 1.0, 0, dst);

                // Dibujar el resultado en el canvas
                cv.imshow('canvasOutput', dst);

                // Llamar al siguiente cuadro (Tiempo Real)
                requestAnimationFrame(process);
            } catch (err) {
                console.log("Esperando flujo de video...");
                requestAnimationFrame(process);
            }
        }
        requestAnimationFrame(process);
    }
</script>
"""

display(HTML(opencv_realtime_sobel))